# 서울특별시 다산콜센터(☎120)의 주요 민원
* 서울특별시 다산콜센터(☎120)의 주요 민원(자주 묻는 질문)에 대한 답변정보
* https://opengov.seoul.go.kr/civilappeal/list

In [ ]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
# request로 가져온 웹사이트의 html 태그를 찾기위해 사용
# 간격을 두고 가져오기 위해 사용

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
# 120 다산 콜센터의 첫 페이지를 먼저 불러와 크롤링할 내용을 봅니다.
base_url = "https://opengov.seoul.go.kr/civilappeal/list?items_per_page=100&page=1"
print(base_url)

https://opengov.seoul.go.kr/civilappeal/list?items_per_page=100&page=1


In [ ]:
# pd.read_html 을 통해 해당 URL의 table 정보를 읽어옵니다.
table = pd.read_html(base_url, encoding="utf-8")
len(table)

1

In [ ]:
table[0]

,번호,제목,생산일,조회수
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177
...,...,...,...,...
95,2375,[서울시민 힐링 프로젝트] 속마음 버스,2014-03-31,3533
96,2374,안산과천선 당고개 급행 (코레일 운영 4호선 급행),2014-03-31,6100
97,2373,여성안심지킴이 집,2014-03-04,3603
98,2372,판게아는 내친구,2014-03-03,568


## 상세정보를 위한 링크정보 수집
* get : 필요한 데이터를 Query String 에 담아 전송
* post : 전송할 데이터를 HTTP 메시지의 Body의 Form Data에 담아 전송

* get 과 post 여부는 브라우저의 네트워크 탭의 Headers > Request Method 를 통해 확인

In [ ]:
# 웹페이지의 결과를 받아옵니다.
response = requests.get(base_url)
response

<Response [200]>

In [ ]:
# status_code를 통한 응답코드 확인 200 == OK
response.status_code

200

In [ ]:
# response.text

## BeautifulSoup 을 통해 html 페이지를 읽기 쉽게 만들기

In [ ]:
# html 태그를 파싱해 올 수 있도록 합니다.
html = bs(response.text, "lxml")
# html

## 상세 정보 수집을 위한 링크 정보 찾기

In [ ]:
# html.table.select("a")

In [ ]:
print(base_url)

https://opengov.seoul.go.kr/civilappeal/list?items_per_page=100&page=1


In [ ]:
# BeautifulSoup 의 select 기능을 통한 링크 태그 찾기
# a 태그 안의 상세 페이지 접근을 위한 링크 번호 수집
# a_list
# a_link_no
# #content > div > div.view-content > div > table > tbody > tr:nth-child(1) > td.data-title.aLeft > a
# html.select("#content > div > div.view-content > div > table > tbody > tr > td > a")
# //*[@id="content"]/div/div[2]/div/table/tbody/tr[1]/td[2]/a
a_list = html.select("#content > div > div.view-content > div > table > tbody > tr > td > a")
a_list[:5]

[<a href="/civilappeal/25670204">다자녀가정 실내 바닥매트 지원</a>,
 <a href="/civilappeal/23194045">[서울산업진흥원] 서울메이드란?</a>,
 <a href="/civilappeal/22904492">광진맘택시 운영(임산부,영아 양육가정 전용 택시)</a>,
 <a href="/civilappeal/22477798">마포 뇌병변장애인 비전센터</a>,
 <a href="/civilappeal/21212235">위드유 서울 직장 성희롱.성폭력 예방센터</a>]

In [ ]:
# str(a_list[0])["href"]

In [ ]:
a_list[0]["href"]

'/civilappeal/25670204'

In [ ]:
a_list[0].get("href")

'/civilappeal/25670204'

In [ ]:
type(a_list[0])

bs4.element.Tag

In [ ]:
# href 값만 가져오기
a_list[0]["href"]

'/civilappeal/25670204'

In [ ]:
# href 값을 / 로 나누기
a_list[0]["href"].split("/")

['', 'civilappeal', '25670204']

In [ ]:
# 내용 번호만 인덱싱
a_list[0]["href"].split("/")[-1]

'25670204'

In [ ]:
# 전처리한 내용을 a_link_no 리스트에 담기
a_link_no = []
for a in a_list:
    a_link_no.append(a["href"].split("/")[-1])

# 앞에서 3개만 슬라이싱
a_link_no[:3]

['25670204', '23194045', '22904492']

In [ ]:
# 리스트컴프리헨션을 사용해서 전처리 하기
a_link_no = [a["href"].split("/")[-1] for a in a_list]
a_link_no[:3]

['25670204', '23194045', '22904492']

In [ ]:
df_table = table[0]
df_table.head(2)

,번호,제목,생산일,조회수
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675


In [ ]:
len(a_link_no), len(df_table)

(100, 100)

In [ ]:
# a_link_no 를 "내용번호" 컬럼을 생성해서 넣어줍니다.
df_table["내용번호"] = a_link_no
df_table

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,156,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
...,...,...,...,...,...
95,2375,[서울시민 힐링 프로젝트] 속마음 버스,2014-03-31,3533,2898630
96,2374,안산과천선 당고개 급행 (코레일 운영 4호선 급행),2014-03-31,6100,2896420
97,2373,여성안심지킴이 집,2014-03-04,3603,2896330
98,2372,판게아는 내친구,2014-03-03,568,2896326


## 특정 페이지를 수집하는 함수 만들기

In [ ]:
def get_one_page(page_no):
    """
    120 주요질문의 특정 페이지 목록을 수집
    """
    # url을 f-string 으로 만들어 줍니다.
    base_url = f"https://opengov.seoul.go.kr/civilappeal/list?items_per_page=100&page={page_no}"
    # requests 를 통해 HTTP 통신으로 url 에 요청을 보내서 결과 값을 받아옵니다.
    response = requests.get(base_url)
    # 위에서 받은 결과값을 재사용합니다. 
    # 테이블을 데이터프레임으로 만들어 줍니다.
    df_table = pd.read_html(response.text)[0]

    # 행의 갯수가 0 이거나 컬럼의 갯수가 4개가 아니라면 데이터가 없는 페이지이기 때문에 예외처리를 해주었습니다.
    if df_table.shape[0] == 0 or df_table.shape[1] != 4:
        return f"{page_no}페이지를 찾을 수 없습니다."
        
    try:
        # 링크 번호 수집을 위해 bs 으로 html 태그를 감싸줍니다.
        html = bs(response.text, "lxml")
        # 테이블 안의 a 태그를 찾습니다.
        a_list = html.select("#content > div > div.view-content > div > table > tbody > tr > td > a")
        # 리스트컴프리헨션을 사용해서 전처리 하기
        # a 태그 안의 href 주소 안의 번호만 가져옵니다.
        a_link_no = [a["href"].split("/")[-1] for a in a_list]
        # 데이터프레임에 내용 링크 번호를 넣어줍니다.
        df_table["내용번호"] = a_link_no
    except:
        # raise Exception(f"{page_no}를 찾을 수 없습니다.")
        return f"{page_no}페이지를 찾을 수 없습니다."
    
    return df_table

In [ ]:
# 없는 페이지도 확인
get_one_page(page_no=1)

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,157,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
...,...,...,...,...,...
95,2375,[서울시민 힐링 프로젝트] 속마음 버스,2014-03-31,3533,2898630
96,2374,안산과천선 당고개 급행 (코레일 운영 4호선 급행),2014-03-31,6100,2896420
97,2373,여성안심지킴이 집,2014-03-04,3603,2896330
98,2372,판게아는 내친구,2014-03-03,568,2896326


In [ ]:
# 함수가 잘 동작하는지 확인
get_one_page(page_no=1000)

'1000페이지를 찾을 수 없습니다.'

## 반복문을 통한 여러 페이지 수집하기

* 게시물이 없을 때까지 수집하기

In [ ]:
import time
# time.sleep을 통해 일정 간격 쉬었다가 가져옵니다.
# 게시물이 없으면 멈춥니다.
page_no = 1
table_list = []
while True:
    print(page_no, end=", ")
    df_one = get_one_page(page_no)
    if type(df_one) == str:
        print("\n수집이 완료되었습니다.")
        break
    table_list.append(df_one)
    page_no += 1
    time.sleep(0.01)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 
수집이 완료되었습니다.


## 데이터 병합하기

In [ ]:
# 여러 페이지의 내용을 하나로 병합합니다.
df = pd.concat(table_list)
df

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,157,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
...,...,...,...,...,...
65,5,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,2007-01-07,374,2894353
66,4,수돗물의 톤당 원가는?,2007-01-07,344,19353862
67,3,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,2007-01-07,326,19699584
68,2,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,2007-01-07,292,2898293


## 데이터 파일로 저장하기

In [ ]:
# 저장할 파일명
file_name = "seoul-120-list.csv"

In [ ]:
# CSV 파일로 저장하기
df.to_csv(file_name, index=False)

In [ ]:
# 저장한 파일 확인
pd.read_csv(file_name)

,번호,제목,생산일,조회수,내용번호
0,2470,다자녀가정 실내 바닥매트 지원,2021-08-17,157,25670204
1,2469,[서울산업진흥원] 서울메이드란?,2021-06-29,675,23194045
2,2468,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",2021-05-13,712,22904492
3,2467,마포 뇌병변장애인 비전센터,2021-03-12,840,22477798
4,2466,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1177,21212235
...,...,...,...,...,...
2465,5,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,2007-01-07,374,2894353
2466,4,수돗물의 톤당 원가는?,2007-01-07,344,19353862
2467,3,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,2007-01-07,326,19699584
2468,2,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,2007-01-07,292,2898293
